In [1]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from statsmodels.api import Logit
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

import seaborn as sns

from sklearn.linear_model import LinearRegression as lm
import statsmodels.formula.api as sm
import statsmodels.api as sm

from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from sklearn.cluster import KMeans,DBSCAN

In [2]:
df_customer=pd.read_csv('movie_customer_cleaned.csv',engine='python')
df_customer.head()

,customer_id,gender,state,age,age_group,married,kids_under12,register_year,drop_flag,drop_year,last_down_date,down_flag,freq_down_weekday
0,C5001,F,NC,51,50s,M,N,2014,Y,2015,12-23-2014,1,3
1,C5002,M,CA,23,20s,S,N,2014,N,.,10-25-2016,1,6
2,C5003,F,TX,39,30s,M,N,2014,N,.,12-21-2018,1,1
3,C5004,F,TX,69,60s+,M,N,2014,N,.,12-30-2014,1,6
4,C5005,F,SC,60,60s+,M,N,2014,N,.,12-09-2018,1,1


In [4]:
df_download=pd.read_csv('movie_download_cleaned.csv',engine='python')
df_download.head()

,customer_id,item_id,down_year,down_date,weekday,inf_year,down_price
0,C5001,I-1010,2014,02-21-2014,6,1.051872,0.946685
1,C5001,I-1014,2014,04-04-2014,6,1.051872,0.946685
2,C5001,I-1017,2014,01-19-2014,1,1.051872,0.946685
3,C5001,I-1028,2014,01-21-2014,3,1.051872,0.946685
4,C5001,I-1035,2014,03-11-2014,3,1.051872,0.899351


In [5]:
download_cnt = pd.DataFrame(df_download["customer_id"].value_counts())
download_cnt=download_cnt.sort_index()

download_cus_id=download_cnt.set_index('customer_id')

download_cnt # 각 customer_id 에 대해서 다운로드 횟수
download_cus_id # 각 

""
customer_id
11
23
50
40
38
...
11
23
5


In [6]:
download_cnt.index
# download_cnt의 인덱스를 customer_id로 인덱스 교체후 그 index값의 값을 불러옴
# 데이터 프레임에 인덱스값을 key로 다시 넣어주기 위해서임

download_cus_id=download_cnt.set_index('customer_id')
download_cus_id.index
# key=customer_id values= 각 고객별 다운로드 수를 데이터 프레임에 담음
download_count=pd.DataFrame({'customer_id':download_cnt.index,'download_cnt':download_cus_id.index})
download_count

,customer_id,download_cnt
0,C5001,11
1,C5002,23
2,C5003,50
3,C5004,40
4,C5005,38
...,...,...
2733,C8273,11
2734,C8274,23
2735,C8275,5
2736,C8276,14


In [7]:
merged_mv_cus=pd.merge(df_customer,download_count,on=['customer_id'],how='left')
merged_mv_cus # df_customer에 customer_id를 기준으로 download_cnt를 join

,customer_id,gender,state,age,age_group,married,kids_under12,register_year,drop_flag,drop_year,last_down_date,down_flag,freq_down_weekday,download_cnt
0,C5001,F,NC,51,50s,M,N,2014,Y,2015,12-23-2014,1,3,11.0
1,C5002,M,CA,23,20s,S,N,2014,N,.,10-25-2016,1,6,23.0
2,C5003,F,TX,39,30s,M,N,2014,N,.,12-21-2018,1,1,50.0
3,C5004,F,TX,69,60s+,M,N,2014,N,.,12-30-2014,1,6,40.0
4,C5005,F,SC,60,60s+,M,N,2014,N,.,12-09-2018,1,1,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3272,C8273,F,SC,49,40s,M,N,2018,N,.,12-18-2018,1,6,11.0
3273,C8274,F,NV,61,60s+,M,N,2018,N,.,12-29-2018,1,7,23.0
3274,C8275,F,CA,59,50s,S,N,2018,N,.,11-23-2018,1,3,5.0
3275,C8276,F,TX,36,30s,S,Y,2018,N,.,12-05-2018,1,5,14.0


In [8]:
for i in range(len(merged_mv_cus)):
    if merged_mv_cus['gender'][i] == 'F':
        merged_mv_cus['gender'][i] = 0
    else:
        merged_mv_cus['gender'][i] =1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [9]:
for i in range(len(merged_mv_cus)):
    if merged_mv_cus['married'][i] == 'S':
        merged_mv_cus['married'][i] = 0
    else:
        merged_mv_cus['married'][i] =1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
for i in range(len(merged_mv_cus)):
    if merged_mv_cus['kids_under12'][i] == 'N':
        merged_mv_cus['kids_under12'][i] = 0
    else:
        merged_mv_cus['kids_under12'][i] =1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [11]:
for i in range(len(merged_mv_cus)):
    if merged_mv_cus['drop_flag'][i] == 'N':
        merged_mv_cus['drop_flag'][i] = 0
    else:
        merged_mv_cus['drop_flag'][i] =1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
merged_mv_cus # gender, married, kids_under_12, drop_flag 더미화

,customer_id,gender,state,age,age_group,married,kids_under12,register_year,drop_flag,drop_year,last_down_date,down_flag,freq_down_weekday,download_cnt
0,C5001,0,NC,51,50s,1,0,2014,1,2015,12-23-2014,1,3,11.0
1,C5002,1,CA,23,20s,0,0,2014,0,.,10-25-2016,1,6,23.0
2,C5003,0,TX,39,30s,1,0,2014,0,.,12-21-2018,1,1,50.0
3,C5004,0,TX,69,60s+,1,0,2014,0,.,12-30-2014,1,6,40.0
4,C5005,0,SC,60,60s+,1,0,2014,0,.,12-09-2018,1,1,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3272,C8273,0,SC,49,40s,1,0,2018,0,.,12-18-2018,1,6,11.0
3273,C8274,0,NV,61,60s+,1,0,2018,0,.,12-29-2018,1,7,23.0
3274,C8275,0,CA,59,50s,0,0,2018,0,.,11-23-2018,1,3,5.0
3275,C8276,0,TX,36,30s,0,1,2018,0,.,12-05-2018,1,5,14.0


In [13]:
# 사용기간 파생변수로 만들어냄. drop_year와 register_year만 사용

for i in range(len(merged_mv_cus)):
    if merged_mv_cus['drop_year'][i]=='.':
        merged_mv_cus.loc[i, 'duration']= 2018 - int(merged_mv_cus['register_year'][i])
    else:
        merged_mv_cus.loc[i, 'duration']= int(merged_mv_cus['drop_year'][i]) - int(merged_mv_cus['register_year'][i])

In [14]:
merged_mv_cus['register_year'] = merged_mv_cus['register_year'].astype('int')

X=merged_mv_cus[['gender','age','married','kids_under12','register_year','drop_flag','duration']] # 독립변수
Y=merged_mv_cus['download_cnt'] # 종속변수

In [15]:
df_cus_cluster1=pd.DataFrame(X)
df_cus_cluster2=pd.DataFrame(Y)
df_cus_cluster=df_cus_cluster1.join(df_cus_cluster2)
df_cus_cluster

# 범주형 자료들 최대한 수치화. state는 36개나 되서 일단 내비둠
# state나 age_group같은 무쓸모 변수들 제외

,gender,age,married,kids_under12,register_year,drop_flag,duration,download_cnt
0,0,51,1,0,2014,1,1.0,11.0
1,1,23,0,0,2014,0,4.0,23.0
2,0,39,1,0,2014,0,4.0,50.0
3,0,69,1,0,2014,0,4.0,40.0
4,0,60,1,0,2014,0,4.0,38.0
...,...,...,...,...,...,...,...,...
3272,0,49,1,0,2018,0,0.0,11.0
3273,0,61,1,0,2018,0,0.0,23.0
3274,0,59,0,0,2018,0,0.0,5.0
3275,0,36,0,1,2018,0,0.0,14.0


In [16]:
# 위의 데이터들 스케일링. scaling할 변수가 너무 많으면 에러 날 때도 있음.

scaler=StandardScaler()
df_scaled=scaler.fit_transform(df_cus_cluster)
df_scaled[:5]

array([[-0.98998001,  0.28505473,  0.75013907, -0.73579364, -1.2016382 ,
         1.99315977, -1.04288868, -1.37501211],
       [ 1.0101214 , -1.38228012, -1.33308614, -0.73579364, -1.2016382 ,
        -0.50171592,  1.34422159, -0.32356803],
       [-0.98998001, -0.42951735,  0.75013907, -0.73579364, -1.2016382 ,
        -0.50171592,  1.34422159,  2.04218116],
       [-0.98998001,  1.35691284,  0.75013907, -0.73579364, -1.2016382 ,
        -0.50171592,  1.34422159,  1.16597776],
       [-0.98998001,  0.82098379,  0.75013907, -0.73579364, -1.2016382 ,
        -0.50171592,  1.34422159,  0.99073708]])

In [18]:
df_scaled_x=df_scaled[:,0:7] # df_scaled_x : 스케일링된 x 변수들
df_scaled_x

array([[-0.98998001,  0.28505473,  0.75013907, ..., -1.2016382 ,
         1.99315977, -1.04288868],
       [ 1.0101214 , -1.38228012, -1.33308614, ..., -1.2016382 ,
        -0.50171592,  1.34422159],
       [-0.98998001, -0.42951735,  0.75013907, ..., -1.2016382 ,
        -0.50171592,  1.34422159],
       ...,
       [-0.98998001,  0.76143611, -1.33308614, ...,  2.17046061,
        -0.50171592, -1.8385921 ],
       [-0.98998001, -0.60816037, -1.33308614, ...,  2.17046061,
        -0.50171592, -1.8385921 ],
       [ 1.0101214 ,  1.05917448, -1.33308614, ...,  2.17046061,
        -0.50171592, -1.8385921 ]])

# 계층형 군집분석

In [ ]:
z_ward=linkage(df_scaled_x,method='ward',metric='euclidean')
# 연결법은 ward, 거리 계산은 euclidean

cluster_hie=fcluster(Z=z_ward, t=5, criterion='distance')
# linkage 함수를 통해 얻은 z_ward 행렬 데이터를 이용해 군집 생성

df_cus_cluster['ClusterHierarchical']=cluster_hie
df_cus_cluster.head()
# 데이터프레임에 계층형 군집 결과 입력

In [ ]:
len(set(df_cus_cluster["ClusterHierarchical"])) # 계층형 군집 결과는 87개 군집

In [ ]:
ds_summary_hc = df_cus_cluster.groupby("ClusterHierarchical").agg(["min", "mean", "max"])

level0 = ds_summary_hc.columns.get_level_values(0)
level1 = ds_summary_hc.columns.get_level_values(1)
ds_summary_hc.columns = level0 + "_" + level1
ds_summary_hc.reset_index(inplace=True)
ds_summary_hc.round(3)

# 머가 크리티컬한걸까..

# K평균 군집분석

In [ ]:
n_cluster= 60     #진명 맘대로 군집 생성 개수
cluster_range = [i+1 for i in range(n_cluster)]
clus_error=[]
for v_n_clus in cluster_range:
    clus = KMeans(v_n_clus)
    clus.fit(df_scaled_x)
    # 각 데이터로부터 가장 가까운 군집 중심점까지 거리 제곱합
    clus_error.append(clus.inertia_)
ds_error=pd.DataFrame({'number of cluster':cluster_range, 'error':clus_error})
ds_error
### 여기선 25개까지 밖에 안했지만 실제론 군집수를 58개로 지정함. 자세한 설명은 다음에서.

군집 개수가 증가함에 따라 중심점까지의 거리는 가까워짐(error = 거리제곱)

In [ ]:
# 군집화 분산 정도의 그래프를 나타낸것. 그래프 라인이 평평해지는 지점을 선택하는 것이 바람직.
plt.figure(figsize=(10,10))
plt.plot(ds_error['number of cluster'],ds_error['error'])
plt.title('sum of square')
plt.xlabel('number of clusters')
plt.ylabel('sum of sqaured distance')

In [ ]:
from sklearn.metrics import silhouette_score
n_clusters = range(2,80)
clusters_inertia = [] 
s_scores = []

for n in n_clusters:
    KM_est = KMeans(n_clusters=n, init='k-means++').fit(df_scaled_x)
    clusters_inertia.append(KM_est.inertia_)    # data for the elbow method
    silhouette_avg = silhouette_score(df_scaled_x, KM_est.labels_)
    s_scores.append(silhouette_avg)

In [ ]:
## 실루엣 계수가 가장 높을때가 군집이 가장 잘 된 지점
##군집수를 58개 지정했던 이유가 아래에 있는 실루엣 계수를 확인해본 결과였음

fig, ax = plt.subplots(figsize=(12,5))
ax = sns.lineplot(n_clusters, s_scores, marker='o', ax=ax)
ax.set_title("Silhouette score method")
ax.set_xlabel("number of clusters")
ax.set_ylabel("Silhouette score")
ax.axvline(68, ls="--", c="red")
plt.grid()
plt.show()

In [ ]:
# 군집계층을 만들어 준건데 이것도 수십여개로 만들어짐

clus=KMeans(68)
clus.fit(df_scaled_x)
cluster_kmeans=[i+1 for i in clus.labels_]
df_cus_cluster['clustr Kmeans']=cluster_kmeans
df_cus_cluster

In [ ]:
ds_summary_km = df_cus_cluster.groupby("clustr Kmeans").agg(["min", "mean", "max"])

level0 = ds_summary_km.columns.get_level_values(0)
level1 = ds_summary_km.columns.get_level_values(1)
ds_summary_km.columns = level0 + "_" + level1
ds_summary_km.reset_index(inplace=True)
ds_summary_km.round(3)

# 머가 크리티컬한걸까..

# DBSCAN 밀도기반 군집분석

In [19]:
## 여기선 밀도기반 군집분석입니다.
## 위의 kmeans 군집분석에 비해 초기 k값은 설정해주지 못하지만 밀도기반이라 성능은 더 좋음

db = DBSCAN(eps=0.3, min_samples=10).fit(df_scaled_x)

# # db.fit(df_scaled_x)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present. 이라고 합니다.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)

Estimated number of clusters: 52


In [20]:
### cluster kmeans와 cluster DBSCAN이라는 변수를 만들어 각 군집에 번호를 붙여준것
### labels라는 변수에 1을 더해준것이 cluster DBSCAN 변수입니다.

# db.fit(df_scaled_x)
labels_db=db.labels_

for (i,val) in enumerate(labels_db):
    if val<0:
        labels_db[i]=0
    else:
        labels_db[i]=labels_db[i]+1

df_cus_cluster['cluster DBSCAN']=labels_db
df_cus_cluster

,gender,age,married,kids_under12,register_year,drop_flag,duration,download_cnt,cluster DBSCAN
0,0,51,1,0,2014,1,1.0,11.0,7
1,1,23,0,0,2014,0,4.0,23.0,1
2,0,39,1,0,2014,0,4.0,50.0,2
3,0,69,1,0,2014,0,4.0,40.0,2
4,0,60,1,0,2014,0,4.0,38.0,2
...,...,...,...,...,...,...,...,...,...
3272,0,49,1,0,2018,0,0.0,11.0,0
3273,0,61,1,0,2018,0,0.0,23.0,0
3274,0,59,0,0,2018,0,0.0,5.0,0
3275,0,36,0,1,2018,0,0.0,14.0,0


In [22]:
ds_summary_db = df_cus_cluster.groupby("cluster DBSCAN").agg(["min", "mean", "max"])


level0 = ds_summary_db.columns.get_level_values(0)
level1 = ds_summary_db.columns.get_level_values(1)
ds_summary_db.columns = level0 + "_" + level1
ds_summary_db.reset_index(inplace=True)
ds_summary_db.round(3).head()

,cluster DBSCAN,age_min,age_mean,age_max,register_year_min,register_year_mean,register_year_max,duration_min,duration_mean,duration_max,download_cnt_min,download_cnt_mean,download_cnt_max
0,0,18,47.105,90,2014,2015.654,2018,0.0,1.278,4.0,1.0,15.425,45.0
1,1,18,47.831,90,2014,2014.000,2014,4.0,4.000,4.0,18.0,33.733,50.0
2,2,18,52.089,78,2014,2014.000,2014,4.0,4.000,4.0,18.0,39.844,56.0
3,3,18,48.333,81,2014,2014.000,2014,4.0,4.000,4.0,20.0,36.447,50.0
4,4,18,38.521,54,2014,2014.000,2014,4.0,4.000,4.0,21.0,39.293,56.0


In [24]:
# dbscan clustering

df_cus_cluster['labels']=labels_db
trace2=go.Scatter3d(
    x=df_cus_cluster['age'],
    y=df_cus_cluster['download_cnt'],
    z=df_cus_cluster['duration'],
    mode='markers',
    marker=dict(
        color=labels_db,
        size=2,
        line=dict(
            color=labels_db,
            width=12
        ),
        opacity=0.8
    )
)

graph_3d=[trace2]

layout2 = go.Layout(
    title = 'customer clustering',
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0  
    ),
    scene = dict(
            xaxis = dict(title  = 'Age'),
            yaxis = dict(title  = 'download_cnt'),
            zaxis = dict(title  = 'duration')
        )
)

fig = go.Figure(data = graph_3d, layout = layout2)
py.iplot(fig)

In [ ]:
d